In [2]:
from cbs_net.net import Net
from cbs_net.stochastic import Stochastic
import time
import csv

In [3]:
sender_code = 731

n = Net()
print("Loading the network...")
start_time = time.time()
n.load_from_file(fnodes='rynek_nodes.txt', flinks='rynek_links.txt', dlm='\t') 
print("Network is loaded in", time.time() - start_time, "sec")
print("Calculating the shortest distances matrix...")
start_time = time.time()
n.sdm = n.floyd_warshall
n.sdm = 0.001 * n.sdm
print("SDM is calculated in", time.time() - start_time, "sec")

Loading the network...
Network is loaded in 0.19981813430786133 sec
Calculating the shortest distances matrix...
SDM is calculated in 498.8136248588562 sec


In [4]:
clients = []
for nd in n.nodes:
    if nd.code < 730:  # 730:
        clients.append(nd)

In [9]:
runs = 5
# 4 rodzaje klientw: sklep, restauracja, bar, inne (hotele, kawiarnie, ...)
probs = {'R': 0.5, 'H': 0.5, 'B': 0.5, 'S': 0.5, 'Ks': 0.5, 'K': 0.5}
weight = 0.03 # average package weight
s_weight = Stochastic(law=1, location=weight, scale=0.25*weight)

In [10]:
f = open("results.txt", 'w')
for i in range(runs):
    n.gen_requests(sender=n.get_node(sender_code), nodes=clients,
                   probs=probs, s_weight=s_weight, verbose=False)
    rts = n.genetic(sender_code=sender_code, requests=n.demand,
                    capacity=0.15, verbose=False)
    td, tw = 0, 0
    for rt in rts:
        tw += rt.transport_work
        td += rt.distance
        # print(rt)
    print(i + 1, len(n.demand), td, tw)
    f.write(str(i + 1) + "\t" + str(len(n.demand)) + "\t" + str(td) + "\t" + str(tw) + "\n")
f.close()

1 356 142.63000000000002 11.952389621528932


KeyboardInterrupt: 